In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidi

In [2]:
from transformers import TrainingArguments
_ = TrainingArguments(output_dir="tmp", eval_strategy="epoch")  # ✅ works on 4.55+

2025-08-18 13:35:26.200848: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755524126.391796      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755524126.447614      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import transformers, evaluate, wandb, torch, optuna
print("transformers:", transformers.__version__)
print("evaluate:", evaluate.__version__)
print("wandb:", wandb.__version__)

from transformers import TrainingArguments
_ = TrainingArguments(output_dir="tmp", eval_strategy="epoch")  # ✅ works on 4.55+



import wandb
wandb.login(key="017a8a1cf1968e847ba05f92a8935af78befe33f")
# =========================
# Optuna HPO for tweet models (local)
# =========================


import os, json, random
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset as TorchDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import joblib


from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    set_seed,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

transformers: 4.52.4
evaluate: 0.4.5
wandb: 0.20.1


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rabeaotman (rabeaotman-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Device: cuda


In [6]:
from pathlib import Path

# Kaggle-friendly paths
PROJECT_DIR = Path("/kaggle/working")  # writable
DATA_PATH   = Path("/kaggle/input/edaaaaaaaaaaaaaaaa/my_eda.csv")  # read-only dataset
RUNS_DIR    = PROJECT_DIR / "outputs" / "hpo_runs"
RUNS_DIR.mkdir(parents=True, exist_ok=True)


TEXT_COL   = "Tweet"        # Text column after your EDA
TARGET_COL = "Sentiment"    # String labels
SEED = 42
set_seed(SEED)
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# --------- Load & encode labels ---------
df = pd.read_csv(DATA_PATH, encoding="latin1")
df = df.rename(columns={'Tweet': 'Original'})
df = df.rename(columns={'normalized_tweet': 'Tweet'})  # this becomes the text we train on
assert TEXT_COL in df.columns and TARGET_COL in df.columns, f"Missing {TEXT_COL}/{TARGET_COL} in CSV."

le = LabelEncoder()
df["label"] = le.fit_transform(df[TARGET_COL])
num_labels = df["label"].nunique()
assert num_labels == 5, f"Expected 5 labels, got {num_labels}"
id2label = {i: name for i, name in enumerate(le.classes_)}
label2id = {v: k for k, v in id2label.items()}

# Single stratified split for HPO (fast). If you prefer CV, see note below.
train_df, eval_df = train_test_split(
    df, test_size=0.2, random_state=SEED, stratify=df["label"]
)

df['Tweet'].iloc[80]

"Love it or hate it , head advice @USER & @USER Blip in our lives but it 's happening ! ? ? Do n't whinge about what you can't do ? ? Dont panic buy as food wont run out ? ? DO spend time with the family ? ? DO use common sense #coronavirus @USER"

In [8]:
# --------- Simple torch Dataset to avoid pyarrow/dependency issues ---------
class TweetDataset(TorchDataset):
    def __init__(self, df, tokenizer, max_length=128, text_col="Tweet"):
        self.texts  = df[text_col].astype(str).tolist()
        self.labels = df["label"].astype(int).tolist()
        self.tok = tokenizer
        self.max_len = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tok(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# --------- Metrics (macro) ---------
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy":  accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "precision": precision.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall":    recall.compute(predictions=preds, references=labels, average="macro")["recall"],
        "f1":        f1.compute(predictions=preds, references=labels, average="macro")["f1"],
    }

import wandb.sklearn as wbsk  # for ROC/PR plots

import inspect
import numpy as np
import torch
import wandb
from pathlib import Path
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, TrainingArguments, Trainer, EarlyStoppingCallback
)

def make_objective(model_name: str):
    def objective(trial):
        # ----- hyperparams to search -----
        lr         = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
        wd         = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
        warmup     = trial.suggest_float("warmup_ratio", 0.0, 0.2)
        max_len    = trial.suggest_categorical("max_length", [96, 128])
        train_bs   = trial.suggest_categorical("train_bs", [8, 16, 32])
        grad_acc   = trial.suggest_categorical("grad_acc", [1, 2, 4])
        label_s    = trial.suggest_float("label_smoothing", 0.0, 0.2)
        grad_ckpt  = trial.suggest_categorical("gradient_checkpointing", [False, True])

        # ----- per-trial W&B run -----
        run = wandb.init(
            project=os.getenv("WANDB_PROJECT", "3-trials-tweet-5label"),
            group=Path(model_name).name.replace("/", "_"),
            name=f"trial-{trial.number}",
            config={
                "model_name": model_name,
                "learning_rate": lr,
                "weight_decay": wd,
                "warmup_ratio": warmup,
                "max_length": max_len,
                "train_bs": train_bs,
                "grad_acc": grad_acc,
                "label_smoothing": label_s,
                "gradient_checkpointing": grad_ckpt,
                "seed": SEED,
            },
            reinit=True,
            settings=wandb.Settings(start_method="thread"),
        )

        # Nice summaries
        wandb.define_metric("loss", summary="min")
        wandb.define_metric("eval/loss", summary="min")
        wandb.define_metric("eval/f1", summary="max")
        wandb.define_metric("eval/accuracy", summary="max")
        wandb.define_metric("eval/precision", summary="max")
        wandb.define_metric("eval/recall", summary="max")

        # ----- tokenizer, datasets, collator -----
        tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        train_ds  = TweetDataset(train_df, tokenizer, max_length=max_len, text_col=TEXT_COL)
        eval_ds   = TweetDataset(eval_df,  tokenizer, max_length=max_len, text_col=TEXT_COL)
        collator  = DataCollatorWithPadding(tokenizer)

        # ----- model -----
        model = AutoModelForSequenceClassification.from_pretrained(
            model_name, num_labels=num_labels, id2label=id2label, label2id=label2id
        ).to(device)

        # ----- dirs & precision -----
        trial_dir = RUNS_DIR / (Path(model_name).name.replace("/", "_") + f"_trial_{trial.number}")
        trial_dir.mkdir(parents=True, exist_ok=True)
        can_bf16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

        # ----- TrainingArguments (compat with 4.52.4 and newer) -----
        TA_params = inspect.signature(TrainingArguments.__init__).parameters
        eval_key = "eval_strategy" if "eval_strategy" in TA_params else "evaluation_strategy"

        args = TrainingArguments(
            output_dir=str(trial_dir),
            **{eval_key: "epoch"},
            save_strategy="epoch",
            logging_strategy="steps",
            logging_steps=50,
            learning_rate=lr,
            weight_decay=wd,
            warmup_ratio=warmup,
            per_device_train_batch_size=train_bs,
            per_device_eval_batch_size=64,
            num_train_epochs=3,
            gradient_accumulation_steps=grad_acc,
            bf16=can_bf16,
            fp16=(not can_bf16) and torch.cuda.is_available(),
            gradient_checkpointing=grad_ckpt,
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",   # your choice
            greater_is_better=True,
            label_smoothing_factor=label_s,
            save_total_limit=1,
            save_safetensors=True,
            report_to=["wandb"],
            run_name=wandb.run.name,
            seed=SEED,
            dataloader_num_workers=2,
            eval_accumulation_steps=4,
        )

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=train_ds,
            eval_dataset=eval_ds,
            tokenizer=tokenizer,
            data_collator=collator,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
        )

        # ----- train & evaluate -----
        trainer.train()
        eval_metrics = trainer.evaluate()
        wandb.log(eval_metrics)

        # ----- robust W&B plots (won't crash the trial) -----
        try:
            preds_out = trainer.predict(eval_ds)
            logits = preds_out.predictions
            y_true = preds_out.label_ids  # ints
            # logits -> probs
            if isinstance(logits, torch.Tensor):
                probs = torch.softmax(logits, dim=-1).detach().cpu().numpy()
            else:
                probs = torch.softmax(torch.tensor(logits), dim=-1).cpu().numpy()
            y_pred = probs.argmax(axis=1)

            # class names (strings)
            try:
                class_names = [model.config.id2label[str(i)] for i in range(num_labels)]
            except Exception:
                class_names = [str(id2label[i]) for i in range(num_labels)]

            # Confusion matrix expects INTS + class_names
            wandb.log({
                "confusion_matrix": wandb.plot.confusion_matrix(
                    y_true=y_true.tolist(),
                    preds=y_pred.tolist(),
                    class_names=class_names
                )
            })

            # ROC & PR
            import wandb.sklearn as wbsk
            wbsk.plot_roc(y_true, probs, labels=class_names)
            wbsk.plot_precision_recall(y_true, probs, labels=class_names)

            # Misclassified table (optional)
            wrong = (y_pred != y_true)
            if getattr(wrong, "any", lambda: False)():
                wr = eval_df.reset_index(drop=True).loc[wrong, [TEXT_COL]].copy()
                wr["true"] = [class_names[i] for i in y_true[wrong]]
                wr["pred"] = [class_names[i] for i in y_pred[wrong]]
                wandb.log({"misclassified": wandb.Table(dataframe=wr.head(200))})
        except Exception as e:
            print("W&B plotting skipped:", e)

        # ----- record best checkpoint & finish -----
        best_ckpt = trainer.state.best_model_checkpoint
        trial.set_user_attr("best_checkpoint", best_ckpt)
        trial.set_user_attr("trial_dir", str(trial_dir))

        # (Optional) upload best checkpoint as artifact
        # art = wandb.Artifact(f"{Path(model_name).name.replace('/','_')}-trial-{trial.number}",
        #                      type="model", metadata={"metrics": eval_metrics})
        # art.add_dir(best_ckpt); wandb.log_artifact(art)

        wandb.finish()
        return float(eval_metrics["eval_accuracy"])
    return objective




# --------- Convenience: run a study for a single model ---------
def run_study(model_name: str, n_trials: int = 10, study_name: str | None = None):
    print(f"\n=== Starting HPO for {model_name} ===")
    objective = make_objective(model_name)
    study = optuna.create_study(direction="maximize", study_name=study_name or Path(model_name).name)
    study.optimize(objective, n_trials=n_trials)

    print("\nBest trial:")
    print("  value (macro-F1):", study.best_trial.value)
    print("  params:", study.best_trial.params)
    print("  best checkpoint:", study.best_trial.user_attrs.get("best_checkpoint"))
    print("  trial dir:", study.best_trial.user_attrs.get("trial_dir"))

    # Persist best params for reproducibility
    best_out = {
        "model_name": model_name,
        "best_value_acurracy": study.best_trial.value,
        "best_params": study.best_trial.params,
        "best_checkpoint": study.best_trial.user_attrs.get("best_checkpoint"),
        "trial_dir": study.best_trial.user_attrs.get("trial_dir"),
        "label_mapping": {int(k): v for k, v in id2label.items()},
    }
    with open(RUNS_DIR / (Path(model_name).name.replace("/", "_") + "_best.json"), "w") as f:
        json.dump(best_out, f, indent=2)
    return study

In [9]:
# 1) BERTweet (large)
study_bt = run_study("vinai/bertweet-large", n_trials=3, study_name="3-trials-bertweet_large_hpo")

[I 2025-08-18 13:38:06,097] A new study created in memory with name: 3-trials-bertweet_large_hpo
wandb: WARNING `start_method` is deprecated and will be removed in a future version of wandb. This setting is currently non-functional and safely ignored.



=== Starting HPO for vinai/bertweet-large ===


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

[I 2025-08-18 15:57:00,416] Trial 1 finished with value: 0.86129699476822 and parameters: {'learning_rate': 2.3715579274751514e-05, 'weight_decay': 5.876764569872929e-06, 'warmup_ratio': 0.11834124638714337, 'max_length': 128, 'train_bs': 16, 'grad_acc': 2, 'label_smoothing': 0.1714776639383804, 'gradient_checkpointing': False}. Best is trial 1 with value: 0.86129699476822.


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.599700,1.587623,0.277771,0.055554,0.200000,0.086955
2,1.589500,1.611471,0.186762,0.037352,0.200000,0.062949
3,1.576600,1.613385,0.186762,0.037352,0.200000,0.062949


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[W 2025-08-18 17:31:22,832] Trial 2 failed with parameters: {'learning_rate': 3.604614767084775e-05, 'w

RuntimeError: [enforce fail at inline_container.cc:626] . unexpected pos 2444723008 vs 2444722900